In [19]:
from getting_started import df_patient, pd

df_patient = df_patient.convert_dtypes()

df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   patient_id     20000 non-null  Int64 
 1   given_name     19560 non-null  string
 2   surname        19575 non-null  string
 3   street_number  19618 non-null  Int64 
 4   address_1      19204 non-null  string
 5   suburb         19788 non-null  string
 6   postcode       19801 non-null  string
 7   state          18010 non-null  string
 8   date_of_birth  17989 non-null  Int64 
 9   age            16003 non-null  Int64 
 10  phone_number   19081 non-null  string
 11  address_2      7893 non-null   string
dtypes: Int64(4), string(8)
memory usage: 1.9 MB


Suppression des patiens avec identifiant dupliqué

In [20]:
df_patient.drop_duplicates(subset={"patient_id"}, inplace=True)

df_patient.set_index("patient_id", inplace=True, verify_integrity=True)

df_patient.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19798 entries, 221958 to 925217
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   given_name     19361 non-null  string
 1   surname        19375 non-null  string
 2   street_number  19422 non-null  Int64 
 3   address_1      19012 non-null  string
 4   suburb         19588 non-null  string
 5   postcode       19602 non-null  string
 6   state          17831 non-null  string
 7   date_of_birth  17802 non-null  Int64 
 8   age            15842 non-null  Int64 
 9   phone_number   18888 non-null  string
 10  address_2      7813 non-null   string
dtypes: Int64(3), string(8)
memory usage: 1.9+ MB


Assainissement de la date de naissance

In [21]:
df_patient.date_of_birth = pd.to_datetime(df_patient.date_of_birth, format="%Y%m%d", errors="coerce")

df_patient.date_of_birth.describe()

count                   17628
unique                  13387
top       1904-04-01 00:00:00
freq                        7
first     1900-01-05 00:00:00
last      1999-12-29 00:00:00
Name: date_of_birth, dtype: object

Assainissement du numéro de rue

In [22]:
df_patient.street_number = df_patient.street_number.replace({0: pd.NA}).astype("Int64")

df_patient.street_number.describe()

count     19416.000000
mean         98.924856
std        2494.481245
min           1.000000
25%          10.000000
50%          24.000000
75%          60.000000
max      342951.000000
Name: street_number, dtype: float64

Inversion quartier - code postal

In [23]:
where = df_patient.suburb.str.contains(r"\d")
df = df_patient.loc[where, ["suburb", "postcode"]]
df.suburb, df.postcode = df.postcode, df.suburb
df.postcode = df.postcode.str.replace(r"[a-z]", "")
df_patient.update(df)

df_patient[where]

,given_name,surname,street_number,address_1,suburb,postcode,state,date_of_birth,age,phone_number,address_2
patient_id,,,,,,,,,,,
810644,juliana,grosvenor,5,connelly pace,port noarlunga south,3023,tas,1999-12-15,<NA>,03 55227740,<NA>
986559,kirra,choi-lundberg,102,centaurus street,naremburn,6168,vic,1926-11-04,27,08 69584599,<NA>
752873,lochlan,blake,258,<NA>,toowoobma,4216,wa,1908-08-21,31,02 84630666,<NA>
902348,isaac,nakoje,19,collier street,brighton,6017,<NA>,1964-04-21,8,02 69439226,<NA>
678110,jaden,green,5,dovey place,oraneg,3185,vic,1915-12-04,23,02 73534391,<NA>
690348,andrew,ryan,20,mainwaring rich circuit,blacktown,3020,wa,NaT,22,<NA>,<NA>
727290,tiffany,loukes,31,leal place,forest hill,4670,nsw,1976-07-06,23,<NA>,pangani
738103,hugi,pascoe,167,leita court,port lincoln,3023,nsw,1904-04-01,<NA>,07 84786511,<NA>
684359,sonia,green,50,kalgoorlie crescent,ashfield,6112,sa,NaT,9,03 46671647,<NA>


Assainissement du code postal

In [24]:
state_postcode = (
    pd.read_csv("state_postcode.csv")
    .convert_dtypes()
    .astype({
        "postcode_min_range": int,
        "postcode_max_range": int,
    })
)

postcode_ranges = pd.arrays.IntervalArray.from_arrays(
    left=state_postcode.postcode_min_range,
    right=state_postcode.postcode_max_range,
    closed="both",
)

validate_postcode = lambda p: postcode_ranges.contains(int(p)).any()

postcode_ranges_per_state = {
    state: pd.arrays.IntervalArray.from_arrays(
        left=postcode.postcode_min_range,
        right=postcode.postcode_max_range,
        closed="both",
    )
    for state, postcode
    in state_postcode.groupby(by="state").agg(tuple).iterrows()
}

validate_state_postcode = lambda s, p: postcode_ranges_per_state[s].contains(int(p)).any()

In [25]:
postcodes = df_patient.postcode.dropna().unique()

to_replace = {p: pd.NA for p in postcodes if not validate_postcode(p)}

df_patient.postcode.replace(to_replace, inplace=True)

df_patient.postcode.describe()

count     19599
unique     2005
top        4350
freq         94
Name: postcode, dtype: object

Assainissement de l'attribut état

In [26]:
from itertools import product
from textdistance import damerau_levenshtein

states = {'act', 'nsw', 'nt', 'qld', 'sa', 'tas', 'vic', 'wa'}
codes = set(df_patient.state.dropna().unique()) - states

df = pd.Series(
    data=[damerau_levenshtein(c, s) for c, s in product(codes, states)],
    index=pd.MultiIndex.from_product([codes, states], names=["code", "state"]),
    name="distance",
)

to_replace = dict(
    df[df == 1].index.to_frame()
    .drop_duplicates(subset="code", keep=False).index
)

codes_na = codes - set(to_replace.keys())

to_replace.update({s: pd.NA for s in codes_na})

df_patient.state.replace(to_replace, inplace=True)

df_patient.state.value_counts()

nsw    6129
vic    4324
qld    3507
wa     1580
sa     1380
tas     508
act     243
nt      134
Name: state, dtype: int64

Nettoyage des états incohérents avec le code postal

In [27]:
df = df_patient[["postcode", "state"]].dropna()

where_incoherent = df.apply(
    lambda x: not validate_state_postcode(x.state, x.postcode),
    axis="columns",
)

df_patient.loc[df[where_incoherent].index, "state"] = pd.NA

df_patient.state.value_counts()

nsw    2087
vic    1086
qld     765
wa      158
sa      108
tas      19
act       6
nt        3
Name: state, dtype: int64

Imputation des états manquants avec le code postal

In [28]:
df = df_patient[df_patient.state.isna()].postcode.dropna()

postcodes = df.unique()

to_replace = pd.Series(
    data={
        postcode: state
        for postcode, state in product(postcodes, states)
        if validate_state_postcode(state, postcode)
    },
).to_dict()

df = df.replace(to_replace).rename("state")

df_patient.update(df)

df_patient.state.value_counts()

nsw    6649
vic    4882
qld    3786
wa     1838
sa     1555
tas     530
act     365
nt      176
Name: state, dtype: int64

Dedup exact matches

In [ ]:
df_dup = df_patient.loc[df_patient.drop(columns=["patient_id"]).duplicated(keep=False)]
df_nodup = df_patient.loc[df_patient.drop(columns=["patient_id"]).duplicated()]

In [ ]:
df = df_dup.merge(df_nodup, on="phone_number")[["phone_number", "patient_id_x", "patient_id_y"]]\
    .dropna().drop(columns=["phone_number"]).set_axis(labels=["patient_id", "dedup_id"], axis="columns")

df_patient_dedup = df.copy()
df.sort_values("dedup_id")

In [ ]:
df = df_dup.merge(df_nodup, on="date_of_birth")[["date_of_birth", "patient_id_x", "patient_id_y"]]\
    .dropna().drop(columns=["date_of_birth"]).set_axis(labels=["patient_id", "dedup_id"], axis="columns")

df_patient_dedup.combine_first(df)